In [1]:
import os
import pandas as pd
import logging
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display, HTML
%matplotlib inline


# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s',
    handlers=[
        logging.FileHandler("merge_data.log"),
        logging.StreamHandler()
    ]
)

print("Libraries imported successfully!")

Libraries imported successfully!


In [2]:
class DataMerger:
    def __init__(self, eda_data_path='data/raw', output_path='data/processed'):
        self.eda_data_path = eda_data_path
        self.output_path = output_path
        self.data_categories = {
            'stock': (['MSFT', 'AMZN', 'GOOGL', 'TSLA'], 'yahoo'),
            'etf': (['SPY', 'QQQ', 'XLK', 'XLV', 'XLF'], 'yahoo'),
            'index': (['^GSPC', '^IXIC', '^DJI', '^RUT'], 'yahoo'),
            'bond': (['^TNX', 'IEF', 'TLT'], 'yahoo'),
            'commodity': (['GC=F', 'CL=F', 'SI=F'], 'yahoo'),
            'reit': (['VNQ', 'SCHH', 'IYR'], 'reits'),
            'crypto': (['BTC', 'ETH', 'LTC'], 'crypto'),
            'fred': (['GDP', 'CPIAUCSL', 'FEDFUNDS', 'UNRATE', 'PPIACO'], 'fred')
        }
        print(f"Data paths initialized:\n- Input: {eda_data_path}\n- Output: {output_path}")
        
    def load_data(self):
        total_loaded = 0
        failed_loads = 0
        
        print("\nData Loading Progress:")
        for category, (items, subfolder) in self.data_categories.items():
            setattr(self, f'{category}_data', {})
            folder_path = os.path.join(self.eda_data_path, subfolder)
            
            for item in items:
                file_path = os.path.join(folder_path, f'{item}.csv')
                data_dict = getattr(self, f'{category}_data')
                
                try:
                    if category == 'fred':
                        data_dict[item] = pd.read_csv(file_path, index_col=0, parse_dates=True)
                    else:
                        data_dict[item] = pd.read_csv(file_path, parse_dates=['Date'])
                    total_loaded += 1
                    print(f"Loaded {item}")
                except FileNotFoundError:
                    print(f"File not found: {file_path}")
                    data_dict[item] = pd.DataFrame()
                    failed_loads += 1
                except Exception as e:
                    print(f"Error loading {item}: {str(e)}")
                    data_dict[item] = pd.DataFrame()
                    failed_loads += 1
        
        print(f"\nLoading Statistics:")
        print(f"Successfully loaded: {total_loaded}")
        print(f"Failed: {failed_loads}")

    def merge_data(self):
        print("Starting data merge process...")
        
        df = pd.DataFrame(index=pd.date_range(start='2010-01-01', end='2023-12-31', freq='D'))
        
        # merge all categories
        for category in ['stock', 'etf', 'index', 'bond', 'commodity', 'reit']:
            data_dict = getattr(self, f'{category}_data')
            for item, item_df in data_dict.items():
                if not item_df.empty:
                    df = df.join(item_df.set_index('Date')['Close'], how='left')
                    df.rename(columns={'Close': f'{item}_Close'}, inplace=True)
                    print(f"Merged {item}")
        
        # handle crypto separately
        for crypto, crypto_df in self.crypto_data.items():
            if not crypto_df.empty:
                df = df.join(crypto_df.set_index('Date')['Price_Close'], how='left')
                df.rename(columns={'Price_Close': f'{crypto}_Close'}, inplace=True)
                print(f"Merged {crypto}")
        
        # handle fred data
        for indicator, indicator_df in self.fred_data.items():
            if not indicator_df.empty:
                df = df.join(indicator_df, how='left')
                print(f"Merged {indicator}")
        
        print("\nCleaning data...")
        df.ffill(inplace=True)
        df.bfill(inplace=True)
        df.dropna(inplace=True)
        
        self.merged_data = df.reset_index().rename(columns={'index': 'Date'})
        print(f"Final dataset shape: {self.merged_data.shape}")
        
        # display sample
        display(HTML("<h3>Sample of Merged Data:</h3>"))
        display(self.merged_data.head())

    def save_merged_data(self):
        os.makedirs(self.output_path, exist_ok=True)
        output_file = os.path.join(self.output_path, 'merged_data.csv')
        self.merged_data.to_csv(output_file, index=False)
        print(f"Merged data saved to: {output_file}")

    def run(self):
        print("Starting data merging pipeline...")
        self.load_data()
        self.merge_data()
        self.save_merged_data()
        print("Pipeline completed successfully!")

In [3]:
# create instance and run
merger = DataMerger()
merger.run()

Data paths initialized:
- Input: data/raw
- Output: data/processed
Starting data merging pipeline...

Data Loading Progress:
Loaded MSFT
Loaded AMZN
Loaded GOOGL
Loaded TSLA
Loaded SPY
Loaded QQQ
Loaded XLK
Loaded XLV
Loaded XLF
Loaded ^GSPC
Loaded ^IXIC
Loaded ^DJI
Loaded ^RUT
Loaded ^TNX
Loaded IEF
Loaded TLT
Loaded GC=F
Loaded CL=F
Loaded SI=F
Loaded VNQ
Loaded SCHH
Loaded IYR
Loaded BTC
Loaded ETH
Loaded LTC
Loaded GDP
Loaded CPIAUCSL
Loaded FEDFUNDS
Loaded UNRATE
Loaded PPIACO

Loading Statistics:
Successfully loaded: 30
Failed: 0
Starting data merge process...
Merged MSFT
Merged AMZN
Merged GOOGL
Merged TSLA
Merged SPY
Merged QQQ
Merged XLK
Merged XLV
Merged XLF
Merged ^GSPC
Merged ^IXIC
Merged ^DJI
Merged ^RUT
Merged ^TNX
Merged IEF
Merged TLT
Merged GC=F
Merged CL=F
Merged SI=F
Merged VNQ
Merged SCHH
Merged IYR
Merged BTC
Merged ETH
Merged LTC
Merged GDP
Merged CPIAUCSL
Merged FEDFUNDS
Merged UNRATE
Merged PPIACO

Cleaning data...
Final dataset shape: (5113, 31)


,Date,MSFT_Close,AMZN_Close,GOOGL_Close,TSLA_Close,SPY_Close,QQQ_Close,XLK_Close,XLV_Close,XLF_Close,...,SCHH_Close,IYR_Close,BTC_Close,ETH_Close,LTC_Close,GDP,CPI,FEDFUNDS,UNRATE,PPI
0,2010-01-01,30.950001,6.6950,15.684434,1.592667,113.330002,46.419998,23.27,31.610001,11.933387,...,12.58,45.810001,457.334015,320.884003,5.05855,14764.61,217.488,0.11,9.8,181.9
1,2010-01-02,30.950001,6.6950,15.684434,1.592667,113.330002,46.419998,23.27,31.610001,11.933387,...,12.58,45.810001,457.334015,320.884003,5.05855,14764.61,217.488,0.11,9.8,181.9
2,2010-01-03,30.950001,6.6950,15.684434,1.592667,113.330002,46.419998,23.27,31.610001,11.933387,...,12.58,45.810001,457.334015,320.884003,5.05855,14764.61,217.488,0.11,9.8,181.9
3,2010-01-04,30.950001,6.6950,15.684434,1.592667,113.330002,46.419998,23.27,31.610001,11.933387,...,12.58,45.810001,457.334015,320.884003,5.05855,14764.61,217.488,0.11,9.8,181.9
4,2010-01-05,30.959999,6.7345,15.615365,1.592667,113.629997,46.419998,23.24,31.299999,12.152721,...,12.58,45.919998,457.334015,320.884003,5.05855,14764.61,217.488,0.11,9.8,181.9


Merged data saved to: data/processed\merged_data.csv
Pipeline completed successfully!
